# EJERCICIO 1: Coeficiente de Clustering en Watts-Strogatz

El modelo Watts-Strogatz es conocido por generar grafos con un alto coeficiente de clustering, una característica común en redes reales.

Tu tarea es implementar una función que estime el coeficiente de clustering promedio de un grafo Watts-Strogatz. Para ello, generarás múltiples grafos con los mismos parámetros y calcularás el promedio de sus coeficientes de clustering.

Parámetros de prueba sugeridos:
  - Número de nodos N = 10
  - Número de vecinos iniciales k = 4
  - Probabilidad de conexión p (variable)
  - Número de repeticiones para la estimación: 100

In [1]:
import numpy as np
import networkx as nx
import random

def cclustering_wsg_solucion(N: int, k: int, p: float, rep: int) -> float:
    # your code here
    valores = []

    for _ in range(rep):
        G = nx.watts_strogatz_graph(N, k, p)
        c = nx.average_clustering(G)
        valores.append(c)

    return float(np.mean(valores))

calculated_clustering = cclustering_wsg_solucion(N=10, k=4, p=0.5, rep=200)
assert abs(calculated_clustering - 0.38) < 0.15, f"Se esperaba un clustering cercano a 0.38, pero se obtuvo {calculated_clustering}"

# EJERCICIO 2: Conteo de Triángulos en el Modelo Random Walk

El modelo Random Walk (caminatas aleatorias) se diseñó, en parte, para abordar  una limitación del modelo Barabási-Albert: la tendencia a generar pocos triángulos y, por ende, bajos coeficientes de clustering. El modelo Random Walk intenta fomentar la formación de triángulos.

Tu tarea es implementar una función que estime el número promedio de triángulos en grafos generados por el modelo `random_walk_model` (provisto anteriormente).
Para ello, generarás múltiples grafos con los mismos parámetros y calcularás el promedio de triángulos.

Parámetros de prueba sugeridos:
  - Número de nodos N = 10
  - Número de conexiones por nuevo nodo m = 2
  - Probabilidad p de conectar a un vecino (variable)
  - Número de repeticiones para la estimación: 100

In [3]:
def random_walk_model(N, m, p):
    # Iniciar un grafo con un pequeño número de nodos y enlaces
    G = nx.complete_graph(m + 1)

    for i in range(m + 1, N):
        # Seleccionar m nodos existentes de manera aleatoria
        random_neighbors = random.sample(list(G.nodes()), 1)
        vecinos = [n for n in G.neighbors(random_neighbors[0])]
        for j in range(m-1):
            if random.random()<p:
                nodos_candidatos = [elemento for elemento in vecinos if elemento not in random_neighbors]
                random_neighbors += random.sample(nodos_candidatos, 1)
            else:
                nodos_candidatos = [elemento for elemento in list(G.nodes()) if elemento not in random_neighbors]
                random_neighbors += random.sample(nodos_candidatos, 1)
        G.add_node(i)
        G.add_edges_from([(i, neighbor) for neighbor in random_neighbors])

    return G

In [4]:
def triangle_RWM_solucion(N: int, m: int, p: float, rep: int) -> float:
    # your code here
    triangulos = []

    for _ in range(rep):
        G = random_walk_model(N, m, p)
        # nx.triangles(G) devuelve un diccionario {nodo: número de triángulos en los que participa}
        total_triangulos = sum(nx.triangles(G).values()) // 3
        triangulos.append(total_triangulos)

    return float(np.mean(triangulos))

avg_triangles_calculated = triangle_RWM_solucion(N=10, m=2, p=0.8, rep=200)
assert avg_triangles_calculated > 6.5 and avg_triangles_calculated < 9.0, f"Se esperaba un promedio de triángulos entre 6.5 y 9.0 para p=0.8, pero se obtuvo {avg_triangles_calculated}"


In [5]:
### BEGIN HIDDEN TEST
import math

# Test 1: N = m+1 (Initial complete graph K_{m+1})
N_eq_mp1, m_eq_mp1, p_eq_mp1, rep_eq_mp1 = 4, 3, 0.5, 50
# Graph is K_4. Triangles = comb(4,3) = 4
expected_triangles_eq_mp1 = math.comb(N_eq_mp1, 3)
calculated_triangles_eq_mp1 = triangle_RWM_solucion(N_eq_mp1, m_eq_mp1, p_eq_mp1, rep_eq_mp1)
assert abs(calculated_triangles_eq_mp1 - expected_triangles_eq_mp1) < 1e-9, f"Para N={N_eq_mp1}, m={m_eq_mp1} (K_{{{N_eq_mp1}}}), se esperaban {expected_triangles_eq_mp1} triángulos, pero se obtuvo {calculated_triangles_eq_mp1:.3f}"


In [6]:
### BEGIN HIDDEN TEST
# Test 2: N=2, m=1 (Graph is K_2)
N_k2, m_k2, p_k2, rep_k2 = 2, 1, 0.5, 50
# Initial graph K_{1+1} = K_2. Loop for new nodes doesn't run. Triangles = 0.
expected_triangles_k2 = 0.0
calculated_triangles_k2 = triangle_RWM_solucion(N_k2, m_k2, p_k2, rep_k2)
assert abs(calculated_triangles_k2 - expected_triangles_k2) < 1e-9, f"Para N={N_k2}, m={m_k2} (K_2), se esperaban {expected_triangles_k2} triángulos, pero se obtuvo {calculated_triangles_k2:.3f}"


In [7]:
### BEGIN HIDDEN TEST
# Test 3: p = 1 (High triangle formation for m > 1)
# Compare with original test (N=10, m=2, p=0.8 -> 6.5 to 9.0)
# For p=1.0, expect more triangles.
N_p1, m_p1, p_p1, rep_p1 = 10, 2, 1.0, 300
calculated_triangles_p1 = triangle_RWM_solucion(N_p1, m_p1, p_p1, rep_p1)
# Based on simulation, for N=10, m=2, p=1, avg triangles is around 9-11.
assert calculated_triangles_p1 > 7.5, f"Para N={N_p1}, m={m_p1}, p={p_p1}, se esperaban más de 8.5 triángulos (actual: {calculated_triangles_p1:.3f}), más que para p=0.8."
